In [15]:
#Mounting the Google Drive
# eta r theke = https://www.projectpro.io/article/python-chatbot-project-learn-to-build-a-chatbot-from-scratch/429
from google. colab import drive
drive. mount( '/content/drive' )
data_root = '/content/drive/MyDrive/chatbot'
#P1ease upload the files in your drive and change the pat

Mounted at /content/drive


In [16]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
nltk.download("punkt")
nltk.download ("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
import glob

data_files = glob.glob('/content/drive/MyDrive/chatbot/*.json')

for data_file in data_files:
  data = json.loads(open(data_file).read())
  print(data)

{'intents': [{'tag': 'hello', 'patterns': ['Hello', 'Hi there', 'Good morning', "What's up"], 'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!'], 'context': ''}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'job', 'patterns': ['What is your job', 'What is your work'], 'responses': ['My job is to make you feel like everything is okay.', 'I work to serve you as well as possible'], 'context': ''}, {'tag': 'age', 'patterns': ['What is your age', 'How old are you', 'When were you born'], 'responses': ['I was born in 2021'], 'context': ''}, {'tag': 'feeling', 'patterns': ['How are you today', 'How are you'], 'responses': ['I am feeling good, you?', 'Very good and you?', "Actually, I'm okay and you?"], 'context': ''}, {'tag': 'good', 'patterns': ['I am good too', 'I feel fine', 'Good !', 'Fine', 'I am good', 'I am great', 'great'], 'responses': ['That is perfect!', "So, ever

In [18]:
#4 Creating data_X and data_Y
words =[]
classes=[]
data_x = []
data_y = []


for intent in data["intents"]:
  for pattern in intent["patterns"]:
      tokens = nltk.word_tokenize(pattern)
      words.extend(tokens)
      data_x.append(pattern)
      data_y.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and # set to ensure no duplicates occur

words = sorted(set(words))
classes = sorted(set(classes))

In [20]:
print(len(classes))
print(data_y[idx])
if data_y[idx] not in classes:
  print("Error: data_y[idx] is not in classes")
  output_row[min(max(classes.index(data_y[idx]), 0), len(classes) - 1)] = 1

15
hello


In [21]:
training = []
out_empty = [0] * len(classes)
classes = sorted(set(data_y))
#creating the bag of words model

for idx, doc in enumerate(data_x):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)

  output_row = list(out_empty)
  output_row[classes.index(data_y[idx])] = 1

  training.append([bow, output_row])

random.shuffle(training)
training = np.array(training, dtype = object)

train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [22]:
#6 The neural network model
model = Sequential()
model.add(Dense(128,input_shape=(len(train_X[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation = 'relu' ))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "Softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
def model_compile(loss, optimizer, metrics):
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

model_compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())
model.fit(x = train_X, y = train_Y, epochs=150, verbose = 1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               7552      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 15)                975       
                                                                 
Total params: 16783 (65.56 KB)
Trainable params: 16783 (65.56 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
2/2

In [38]:
def clean_text(text):
  tokens=nltk.word_tokenize(text)
  tokens=[lemmatizer.lemmatize(word)for word in tokens]
  return tokens
def bag_of_words(text,vocab):
  tokens=clean_text(text)
  bow=[0]*len(vocab)
  for w in tokens:
    for idx,word in enumerate(vocab):
      if word==w:
        bow[idx]=1
  return np.array(bow)

def pred_class(text,vocab,labels):
  bow=bag_of_words(text,vocab)
  result=model.predict(np.array([bow]))[0]
  thresh=0.5
  y_pred=[[indx,res] for indx,res in enumerate(result) if res>thresh]
  y_pred.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list,intents_json):
  if len(intents_list)==0:
    result="Sorry! I don't understand"
  else:
    tag=intents_list[0]
    list_of_intents=intents_json["intents"]
    for i in list_of_intents:
      if i["tag"]==tag:
        result=random.choice(i["responses"])
        break
  return result

In [27]:
def pred_class(sentence, words, classes):
    # Function to predict the class of the sentence

    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    bag = [1 if word in words else 0 for word in sentence_words]
    return np.array(bag)

def get_response(intents_list, data):
    # Function to get the bot's response

    tag = intents_list[0]['intent']
    list_of_intents = data['intents']
    result = ""
    for i in list_of_intents:
        if i['intent'] == tag:
            result = i['response']
            break
    return result

In [39]:
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
  message=input("")
  if message=="0":
    break
  intents=pred_class(message,words,classes)
  result=get_response(intents,data)
  print(result)

Press 0 if you don't want to chat with our ChatBot.
hi
1/1 [==============================] - 0s 107ms/step
Hello
hey
1/1 [==============================] - 0s 24ms/step
That is perfect!
how are you
1/1 [==============================] - 0s 23ms/step
I am feeling good, you?
bad
1/1 [==============================] - 0s 24ms/step
I hope you will feel better !
0
